# Part 1: Database construction and cleaning

## Executive summary: 

**Objective:**

* My objective for this project is to construct a model that predicts whether an Australian Federeal Electorate will tend to vote right or left. This model will use a variety of demographic and some voting statistics to find which features can predict left or right leaningness. 


* In order to do this, I must collect comprehensive demographic and election data. The best sources for these are the Australian Electoral Commission (AEC) for election data, and the Australian Bureau of Statistics (ABS) for census data. This data will also need to be grouped by each electorate and will need to be joined by the election year and each individual electorate. 


* Because the data will be very fragmented and made up of many individual CSVs, I will need to store it in my own data base and organise it so every individual category and table relates to each other by its individual electorate identifier and its relevant election year.


* This will be done via PostgreSQL, and I'll be using psycopg2 to connect and write functions that will automatically clean my CSVs and upload them to my data base.

**Getting the data and data sources:**

* **Election data** - https://results.aec.gov.au/ 

    * Election data was relatively easy to download, with only 5 CSVs per election, in very easy to access downloadable files. In total there were 35 CSVs, 5 categories for the past 7 election years.

* **Census data** - https://www.abs.gov.au/websitedbs/censushome.nsf/home/historicaldata

    * Census and demographics data was much harder to download and build. Firstly, raw census data is only available for the 2016 and 2011 census and nothing before. It was crucial to my goals to have as many demographics data points as possible, so I wanted to encapsulate as much census data as possible.
    
    * Unfortunately, it was impossible to have any sort of meaningful statistics for any census data before 2006, so I had to settle with three demographics data points, the 2016, 2011, and 2006 Census'. 
    
    * Because 2006 did not have any available raw data, I had to resort to using the ABS' table builder. Table builder essentially allows the user to access the ABS' database and construct your own tables and download them as a CSV. 
    * Unfortunately, there was no way I could automate the process of building every individual table for every demographic category I wanted, so I had to manually construct 81 CSVs using the ABS' table builder. 27 categories for every census across the 3 different census years
    
## Data base construction:

* With the CSVs now downloaded and constructed, it was time to build the database that will store all the data from all the CSVs. 

**Challenges**

* The main challenges presented here are mostly to with the Census data. Some of the Census questions will differ slightly from one another, so I will need to go through every single CSV and see if any of the column questions aren't the same, and if they aren't, I will need to align them so they're all identical. 


* Similarly, I will also need to align income, family income, rent and mortgage brackets. Since the Census' span across a decade, some of the financial figures will need to be adjusted for inflation, and some of the income brackets will need to be rolled over to align with one another.


* The other thing I will need to deal with is turning all the raw counts into proportions. So I will need to write a function that automatically converts any count into a proportion (dividing it by the total figure). This will need to be done for every single cell in every single CSV.


* The column names will need to be adjusted so they are suitable for SQL naming conventions, this includes removing any special characters and not starting any column with a number.


* Because there are 7 election years and only 3 census years, I will need to find a way to align them. To do this I will interpolate the census data using a carried annual growth rate, calculated from the growth over 5 years between each census. While not perfect, this will essentially fill in the gaps and align the census data to each election year, making a much more accurate comparison.

**Below I will begin with the database construction process:**

In [2]:
#Importing various modules that I need for my data base construction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#This module will allow me to interact and execute queries via python on my postgresql data base
import psycopg2
#This module allows me to read in and edit multiple files in a directory
import os
%matplotlib inline

import csv
from sqlalchemy import create_engine
#My custom script containing all methods I will be using for wrangling and sorting
from cowboy import wrangler as wr

In [2]:
#Adjusting display options for the data frame
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows',20)

**Importing election data from the AEC and putting it into SQL:**

* As I am just starting out, I will manually construct the SQL tables so that data can be inserted into them. This is only for the AEC data, as there is not a lot of it and it's pretty straight forward with the naming conventions. Minimal manipulation is needed for the actual data, so doing it manually is enough for me now.

In [9]:
#Connecting to my local database on Postgresql
conn_str = "host='localhost' \
dbname='Election_DB' \
user='postgres' \
password='1234'"

conn = psycopg2.connect(conn_str)
cur = conn.cursor()

In [4]:
#Manually creating the scaffolding tables so I can insert the csvs later

#Creating a string of a SQL query that names the table and assigns the data type to each column manually
exe_enrolment = """
CREATE TABLE GeneralEnrolment (
    DivisionID int,
    DivisionNm text,
    StateAb text,
    CloseOfRollsEnrolment int,
    NotebookRollAdditions int,
    NotebookRollDeletions int,
    ReinstatementsPostal int,
    ReinstatementsPrePoll int,
    ReinstatementsAbsent int,
    ReinstatementsProvisional int,
    Enrolment int,
    Election_Year int)
    """

exe_membelect = """
CREATE TABLE HouseMembersElected (
    DivisionID int,
    DivisionNm text,
    StateAb text,
    CandidateID int,
    GivenNm text,
    Surname text,
    PartyNm text,
    PartyAb text,
    Election_Year int)
    """

exe_tpp = """
CREATE TABLE HouseTpp (
    DivisionNm text,
    DivisionID int,
    StateAb text,
    PartyAb text,
    LNCVotes int,
    LNCPercentage float,
    ALPVotes int,
    ALPPercentage float,
    TotalVotes int,
    Swing float,
    Election_Year int)
    """

exe_turnout = """
CREATE TABLE HouseTurnout (
    DivisionID int,
    DivisionNm text,
    StateAb text,
    Enrolment int,
    Turnout int,
    TurnoutPercentage float,
    TurnoutSwing float,
    Election_Year int)
    """
exe_votecount = """
CREATE TABLE HouseVotesCounted (
    DivisionID int,
    DivisionNm text,
    StateAb text,
    Enrolment int,
    OrdinaryVotes int,
    AbsentVotes int,
    ProvisionalVotes int,
    PrePollVotes int,
    PostalVotes int,
    TotalVotes int,
    TotalPercentage float,
    Election_Year int)
    """

#Making a list of the queries for later
exe_list = [exe_enrolment,exe_membelect,exe_tpp,exe_turnout,exe_votecount]

In [5]:
#Defining the file name categories and years manually so they can be uploaded corresponding to each table
election_years = ['2019','2016','2013','2010','2007','2004']
file_names = ['GeneralEnrolmentByDivisionDownload','HouseMembersElectedDownload','HouseTppByDivisionDownload',
              'HouseTurnoutByDivisionDownload','HouseVotesCountedByDivisionDownload']

In [5]:
#Going through all the files in my directory and uploading them corresponding to each table
election_years = ['2019','2016','2013','2010','2007','2004']
for exes,names in zip(exe_list,file_names):
    #Executes the query in the pre defined query list
    cur.execute(exes)
    for year in election_years:
        with open('./Database/AEC/' + names + '-' + year + '.csv', 'r') as f:
            next(f)
            #Uploads the data to the corresponding table name
            cur.copy_from(f, names.replace('Download','').replace('ByDivision',''), sep=',')
            conn.commit()
    #Finalises the changes and uploads them to the database
    conn.commit()

* The first five tables have now been uploaded into SQL, a relatively simple and straightforward process. With the Census data, this process will become far more complex not only because of the amount of CSVs, but also how messy the data is.

## Cleaning and uploading the Census data:

* Now for the hard part. For this section I will be importing a lot of functions that I have written from a module I wrote. It simply contains a series of functions that will clean the CSVs and upload them to the PostgreSQL database.

In [3]:
#The reference table is so each of the electorate rows can be assigned a unique DivisionID so they can be aligned
ref_table = pd.read_csv('./Database/AEC/All_reftable.csv')
ref_table.rename(columns={'DivisionNm':'CED','StateAb':'CED_State'},inplace=True)
ref_table.drop_duplicates(subset='CED',inplace=True)
ref_table['CED'] = ref_table['CED'].apply(lambda x:x.lower())
ref_table

,CED,CED_State,DivisionID,Year
0,adelaide,SA,179,2019
1,aston,VIC,197,2019
2,ballarat,VIC,198,2019
3,banks,NSW,103,2019
4,barker,SA,180,2019
...,...,...,...,...
442,throsby,NSW,150,2013
443,wakefield,SA,191,2013
681,kalgoorlie,WA,241,2007
693,lowe,NSW,129,2007


* The reference table above contains every electorate from every election year since 2004 and contains their unique division ID. The purpose of this table is so I can align the electorate names from the Census data to the divisionID from the AEC. This will make it far easier to join later on.

In [4]:
#Iterates through the entire directory, and outputs it to the 'clean' directory
directory = './Database/ABS/table_builder'
for filename in os.listdir(directory):
    if '.DS_Store' not in filename:
        #Applies the CSV cleaner function to every file in the table builder directory (refer to docstring)
        wr.csv_cleaner(os.path.join(directory,filename),filename)

In [4]:
#Defining the directory that we want to upload to SQL from
#Clean directory is the cleaned files after going through wr.csv_cleaner
clean_directory = './Database/ABS/clean_table_builder/'

#This is all the category names for every file in the clean directory
categories = set([filename.split('_')[1] for filename in os.listdir(clean_directory) if '.DS_Store' not in filename])

In [5]:
#Uploading the CSVs into SQL for all files in the clean directory
for category in categories:
    wr.sql_upload(category,clean_directory)

* The process of getting the raw CSVs was relatively straight forward. You take the raw CSVs, put them through the CSV cleaner function, output them to a new directory, then upload them to its respective SQL table. 


* The underlying functions were much more difficult and complicated, and took a lot of patience to get right. But for now, the actual Census data is now in the SQL data base.

## Interpolating new census data:

* With the 2006, 2011, and 2016 Census' now in my SQL database, it is time to interpolate more data for the 2004, 2007, 2010, 2013, and 2019 elections. The only aligning Census and election year was 2016, so I will not have to interpolate for that year. 


* The interpolation process is a lot more difficult than the simple CSV cleaning process above. I will run through the steps to give an overview of how it will work:

    1. From the original uncleaned table builder files, run the 'csv_interpolate_clean' function. This custom function is a modification of the original CSV cleaner function, with the difference being it does not take a proportion.
    
    2. Once the files have gone through the preliminary clean, it is time to put them through the 'interpolater'. This is function works by taking in a specified year that you want to interpolate to (e.g. 2007), and taking in the upper bound and lower bound Census years. The reason for taking in the upper/lower bound is so it can calculate the growth rate over 5 years, and calculate a Carried Annual Growth Rate on a year basis. For the 2007 example, it will calculate the 5 year growth rate from 2006-2011, calculate the CAGR per year, then apply that for one year growth for 2007. For 2010, it will apply the CAGR from 2006-2011 and apply that for three years growth (CAGR^3).
    
    3. The interpolater, once it has calculated the CAGR between the lower and upper bound Census years, will apply the growth to every single cell in every single CSV. It is now left with a series of counts and not proportions.
    
    4. Finally, the interpolater will take the interpolated counts and transform them into proportions, identical to the original Census years' cleaned CSVs.

In [4]:
#Cleaning the original table_builder csvs to be set up so they can be interpolated with non census years later on
#Defining the directory from where we will be cleaning the files
new_dir = './Database/ABS/table_builder/'
for file in os.listdir(new_dir):
    #Applying the interpolate_clean method from the wrangler class (refer to docstring)
    wr.csv_interpolate_clean(os.path.join(new_dir,file),file)

In [4]:
#Defining the directory with the ready to be interpolated files
interpolate_dir = './Database/ABS/to_interpolate/'

#Separating the files into their respective census years, this will make it easier to organise
census_06 = [file for file in os.listdir(interpolate_dir) if '2006' in file]
census_11 = [file for file in os.listdir(interpolate_dir) if '2011' in file]
census_16 = [file for file in os.listdir(interpolate_dir) if '2016' in file]

#Setting a list of the 27 category names of the census
names = set([name.split('_')[1] for name in os.listdir(interpolate_dir) if '.DS_Store' not in name])

In [5]:
#Now filtering each file based on name and census year
for name in names:
    c_06 = [file for file in census_06 if name == file.split('_')[1]][0]
    c_11 = [file for file in census_11 if name == file.split('_')[1]][0]
    c_16 = [file for file in census_16 if name == file.split('_')[1]][0]
    #Interpolating for all the election years (bar 2016 since that is already aligned)
    for year in ['2004','2007','2010','2013','2019']:
        #Applying the interpolater based on the lower/upper bounds if it is less than 2011
        if float(year) < 2011:
            wr.interpolater(c_06,c_11,interpolate_dir,year)
        #Applying it with the low/up bounds being 2011 and 2016
        else:
            wr.interpolater(c_11,c_16,interpolate_dir,year)

* Now that the interpolation is done, it is simply a matter of uploading the interpolated CSVs into their respective SQL tables on my database. 


* To do this, I'm using a slightly modified version of my original SQL uploader function. The original SQL uploader function had to construct the empty table first before inputting the CSV data. With this one, since the empty SQL tables are already built, I just need to simply upload the data corresponding to the correct category name.

In [6]:
#Defining the interpolated directory
interpolated_directory = './Database/ABS/interpolated/'
#Getting a list of category names from the directory
categories_int = set([filename.split('_')[1] for filename in os.listdir(interpolated_directory) if '.DS_Store' not in filename])

In [7]:
#Uploading the interpolated data to the database with the already defined tables
for category in categories_int:
    #Refer to docstring for more info on the wr.sql_upload_int method
    wr.sql_upload_int(category,interpolated_directory)

* The database is now complete, there should be demographics and election data for every single election year spanning 2004-2019. I am estimating about 900 rows of data, approximately 150 per election year. The data frame will be quite wide and relatively shallow, but given the amount of data I had at my disposal, I am happy with the results so far.

## Joining tables to construct the final dataframe:

* With my database now fully constructed and easily accessible on SQL, it's time to join them all together to make a workable dataframe for my Exploratory Data Analysis and modeling. 


* In all, I will be joining 32 different SQL tables together. They will be joined by both their respective unique DivisionID and their respective election year. So the overall join will be on two columns over 32 tables.


* This was a daunting task, but once I found the correct syntax, I could easily write a function that'd generate an extremely long SQL query to join every table on those two columns.


* The process was as follow:

    1. Separate the election data and census data. I will be making one table for all the AEC data, and one for all the ABS data via joining them through SQL.
    2. With my SQL joiner function (refer to docstring), I will iterate through every single table name, and do an inner join on both DivisionID and election year. This will make one big table containing all the census data and interpolated census data for every election since 2004. This process will also be the same for the election data
    3. Once I have those two tables I will read them into Pandas and do a simple pandas merge on the same two columns. 
    4. Done! After that merge, the table is constructed and I have my finished data frame.

In [10]:
#Grabbing all the table names from my SQL database
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
    
#Putting them into a list for use below
sql_tables = [table[0] for table in cursor.fetchall()]

In [11]:
#Splitting the tables into election and census data, this will make it much easier to join
election_tables = ['housetpp','housememberselected','houseturnout','housevotescounted','generalenrolment']

census_tables = [table for table in sql_tables if table not in election_tables]

In [12]:
#Using the wr.sql_join method I am making a query that joins all the columns from every single table
election_q = wr.sql_join(election_tables,'election')
#Reading it into a dataframe
elec = pd.read_sql(election_q,con=conn)
#Removing duplicate columns
elec = elec.loc[:,~elec.columns.duplicated()]
#Making a new 'year' column for easier joining
elec['year'] = elec['election_year']

census_q = wr.sql_join(census_tables,'census')
#Reading it into a dataframe
cens = pd.read_sql(census_q,con=conn)
#Removing duplicate columns
cens = cens.loc[:,~cens.columns.duplicated()]
#Making a new 'year' column for easier joining
cens['year'] = cens['census_year']

In [13]:
#Now with the two tables defined, I will merge them on an inner join with aligning division id and year
demo_df = elec.merge(cens,on=['divisionid','year'])
#The finished product, voila!
demo_df

,divisionnm,divisionid,stateab,partyab,lncvotes,lncpercentage,alpvotes,alppercentage,totalvotes,swing,...,employer_government_includes_defence_housing_authority_pct,employer_other_employer_pct,housing_co_operative_community_church_group_pct,owned_outright_pct,owned_with_a_mortgage_pct,being_purchased_under_a_rent_buy_scheme_pct,rented_pct,being_occupied_rent_free_pct,being_occupied_under_a_life_tenure_scheme_pct,other_tenure_type_pct
0,Adelaide,179,SA,ALP,44819,41.82,62362,58.18,107181,0.12,...,0.062538,0.657678,1.422633,23.297441,23.705533,0.057609,32.926855,0.761181,0.880346,0.323751
1,Aston,197,VIC,LP,60180,60.13,39910,39.87,100090,2.72,...,0.000000,0.175370,0.306101,28.943640,36.751440,0.068675,20.402274,0.501577,0.859395,0.459208
2,Ballarat,198,VIC,ALP,40068,39.02,62615,60.98,102683,-3.62,...,0.042602,0.296630,0.622956,27.043727,29.219009,0.036482,21.820657,0.660488,0.291292,0.396658
3,Banks,103,NSW,LP,51609,56.26,40121,43.74,91730,4.82,...,0.078412,0.338183,0.662237,29.647115,28.422682,0.039335,28.825386,0.673301,0.337973,0.284562
4,Barker,180,SA,LP,72851,68.94,32815,31.06,105666,5.07,...,0.297276,1.336800,0.652348,27.535345,24.324854,0.039489,20.001748,1.301663,0.855056,0.437370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,Watson,251,NSW,ALP,24029,34.86,44899,65.14,68928,2.17,...,0.065619,0.460131,0.398981,32.446206,21.630599,0.294440,29.506493,0.728206,0.177017,0.229964
872,Wentworth,152,NSW,LP,40847,55.48,32777,44.52,73624,-2.38,...,0.088357,0.296624,0.296051,21.849481,15.382495,0.069304,31.466465,0.898127,0.266259,0.269678
873,Werriwa,153,NSW,ALP,31570,40.69,46012,59.31,77582,-0.82,...,0.171476,0.272215,0.380005,20.458323,39.271324,0.215366,25.780885,0.513531,0.288302,0.190053
874,Wide Bay,178,QLD,NP,51489,62.89,30388,37.11,81877,2.99,...,0.210800,0.645038,0.653188,32.090607,23.402774,0.188344,21.027154,1.137351,0.345970,1.008388


In [14]:
#Outputting the finished product to a csv file which will be read in for more cleaning later
demo_df.to_csv('./Database/CED_data.csv',index=False)

* The finished product, is a 876 row deep and 290 column wide data frame. The excellent thing about constructing your own dataframe from the ground up is that there are no missing values, and that all the values that will be modeled are already on the exact same scale. Having everything in proportion means I do not have to standardise any of the figures.


* The amount of time and work that was put in to making this dataframe will be worth it once it comes to the modeling, as there will be very little I need to do to make it workable. 


* Over the next two sections I will do a little bit of cleaning for labeling and readability, but the dataframe in the form its in now, needs very little work.